## Setup

In [5]:
from dotenv import load_dotenv
from utils import chat_interface
from agentic.tools import db_tools

In [7]:
import os
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from agentic.agents.agents import (
    SupervisorAgent,
    RAGAgent,
    SubscriptionAgent,
    ReservationAgent
)
from agentic.tools.mcp_server import get_vector_store, populate_vector_db

load_dotenv()

# Initialize Shared Resources
vector_store = get_vector_store()
print("✅ Environment and Resources Loaded")

✅ Environment and Resources Loaded


In [2]:
load_dotenv()

False

In [8]:
#initialize vector DB
populate_vector_db()

🔄 Indexing Knowledge Base...
❌ Error populating vector DB: [WinError 5] Access is denied: 'data/vector_store\\443e568e-c73d-4fef-bb9a-dcb4cb74eb14'


# Testing MCP Server

In [16]:
from agentic.tools.mcp_server import *

In [15]:
import nest_asyncio

nest_asyncio.apply()

import os
import sys
import io

from mcp import ClientSession, StdioServerParameters

from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools

from agentic.tools.mcp_config import MCP_SERVERS
from agentic.tools.mcp_server import mcp

nest_asyncio.apply()

# UPDATE: Import from the adapter package
server_params = StdioServerParameters(
    command=sys.executable,
    args=["mcp_server.py"],
)

# Ensure we can import from local modules
#sys.path.append(os.getcwd())


async def test_mcp_connection():
    print("🚀 Starting MCP Client Test (via langchain-mcp-adapters)...")

    errlog = sys.stderr
    try:
        sys.stderr.fileno()
    except (io.UnsupportedOperation, AttributeError):
        errlog = open(os.devnull, 'w')

    async with stdio_client(server_params, errlog=errlog) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # Get tools
            tools = await load_mcp_tools(session)

            try:

                print("\n🔌 Connecting to server...")
                print("\n--- Testing Subscription Service ---")
                for t in tools:
                    print(f"   - {t.name}: {t.description[:50]}...")

                assert any(t.name == "lookup_customer" for t in tools), "❌ Missing 'lookup_customer'!"

                print("\n--- Testing Reservation Service ---")
                assert any(t.name == "get_available_experiences" for t in tools), "❌ Missing 'get_available_experiences'!"

                print("\n--- Testing Knowledge Service ---")
                assert any(t.name == "search_knowledge_base" for t in tools), "❌ Missing 'search_knowledge_base'!"

            except Exception as e:
                print(f"\n❌ Test Failed: {e}")
                import traceback
                traceback.print_exc()
            finally:
                print("\n🔌 Closing connections...")
                #await client.__aexit__(None, None, None)
                print("✅ Done.")





In [16]:
nest_asyncio.apply()
await test_mcp_connection()

🚀 Starting MCP Client Test (via langchain-mcp-adapters)...

🔌 Connecting to server...

--- Testing Subscription Service ---
   - lookup_customer: Retrieves customer profile by email (ID, Name, Blo...
   - get_user_subscription: Fetches subscription details (Tier, Status, Quota)...
   - cancel_subscription_action: Cancels a user's subscription immediately....
   - get_available_experiences: Lists upcoming experiences that have available slo...
   - get_user_reservations: Lists existing reservations for a user....
   - create_reservation_action: Books an experience for a user if slots are availa...
   - search_knowledge_base: Semantic search for knowledge base articles....

--- Testing Reservation Service ---

--- Testing Knowledge Service ---

🔌 Closing connections...
✅ Done.


In [7]:
import asyncio
loop = asyncio.get_event_loop()
asyncio.run_coroutine_threadsafe(test_mcp_connection(), loop)


<Future at 0x20123cda5d0 state=pending>

🚀 Starting MCP Client Test (via langchain-mcp-adapters)...


# Testing Single Agents

### Testing RAG aka Knowledge base Agent

In [9]:
print("--- Testing RAG Agent ---")
rag_agent = RAGAgent(vector_store).get_graph()

# Test Query
response = rag_agent.invoke({
    "messages": [HumanMessage(content="What is the cancellation policy for premium events?")]
})

print(f"User Question: {response['messages'][0].content}")
print(f"Agent Answer: {response['messages'][-1].content}")

--- Testing RAG Agent ---
User Question: What is the cancellation policy for premium events?
Agent Answer: The cancellation policy for premium events is as follows: 
- 100% refund if cancelled more than 48 hours before the start.
- 50% refund if cancelled between 24 to 48 hours before the start.
- No refund if cancelled less than 24 hours before the start.


# Testing Subscription Agent

In [10]:
print("\n--- Testing Subscription Agent ---")
sub_agent = SubscriptionAgent().get_graph()

# Note: We simulate a user providing their email for context
# Ensure this email exists in your seeded DB (e.g., from 01_external_db_setup)
test_email = "alice.kingsley@wonderland.com"

response = sub_agent.invoke({
    "messages": [
        HumanMessage(content=f"My email is {test_email}. What is my current subscription status?")
    ]
})

print("Conversation History:")
for msg in response['messages']:
    if msg.type == "human":
        print(f"User: {msg.content}")
    elif msg.type == "ai":
        if msg.tool_calls:
            print(f"Agent (Tool Call): {msg.tool_calls[0]['name']}")
        else:
            print(f"Agent: {msg.content}")
    elif msg.type == "tool":
        print(f"Tool Output: {msg.content}")


--- Testing Subscription Agent ---


TypeError: 'FunctionTool' object is not callable

In [11]:
from agentic.tools.mcp_server import (
    lookup_customer,
    get_user_subscription,
    cancel_subscription_action,
    get_available_experiences,
    get_user_reservations,
    create_reservation_action
)

In [14]:
lookup_customer.execution("bob.stone@granite.com")

TypeError: 'NoneType' object is not callable

## Run

In [ ]:
# TODO: Develop your agents under `agentic/agents`
# TODO: Develop your tools under `agentic/tools`
# TODO: Modify `agentic/workflow` in order to orchestrate your agents

In [ ]:
# IDEALLY YOUR ONLY IMPORT HERE IS:
# from agentic.workflow import orchestrator

from agentic.workflow import orchestrator

In [4]:
chat_interface(orchestrator, "1")

User: Hi
Assistant: Hello! How can I assist you today?
User: q
Assistant: Goodbye!


In [5]:
list(orchestrator.get_state_history(
    config = {
        "configurable": {
            "thread_id": "1",
        }
    }
))[0].values["messages"]

[HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='43c56100-7a4e-4ff0-adb7-1fbbfdac82e3'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 60, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C1loRWd5jRqktu5Fut6YZAZsTdc6S', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0ab48c4a-6130-4dc9-96cd-271e2be7b7c8-0', usage_metadata={'input_tokens': 60, 'output_tokens': 10, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]